## Import libraries etc.

In [ ]:
import pandas as pd
import csv
import re
import numpy as np
from ccdc.io import EntryReader
from ccdc.io import CrystalReader
csd_entry_reader = EntryReader('CSD')
csd_crystal_reader = CrystalReader('CSD')

Again, the three cells below are form the previous notebook, making a dataframe and parsing it.

In [ ]:
store = []
i = 0
for entry in csd_entry_reader:
    store.append(((((entry.identifier, entry.formula, entry.crystal.cell_volume, entry.crystal.z_value, entry.crystal.z_prime)))))
    i += 1
    if i == 1000:
        break
headers = ["Refcode", "Molecular Formula", "Cell volume", "Z", "Z prime"]

df = pd.DataFrame(data = store, columns = headers)
df = df[df['Z'] > 0]


In [ ]:
def molparsed(row):
    baseformula = {"H":0,"He":0,"Li":0,"Be":0,"B":0,"C":0,"N":0,"O":0,"F":0,"Ne":0,
    "Na":0,"Mg":0,"Al":0,"Si":0,"P":0,"S":0,"Cl":0,"Ar":0,"K":0,"Ca":0,
    "Sc":0,"Ti":0,"V":0,"Cr":0,"Mn":0,"Fe":0,"Co":0,"Ni":0,"Cu":0,"Zn":0,
    "Ga":0,"Ge":0,"As":0,"Se":0,"Br":0,"Kr":0,"Rb":0,"Sr":0,"Y":0,"Zr":0,
    "Nb":0,"Mo":0,"Tc":0,"Ru":0,"Rh":0,"Pd":0,"Ag":0,"Cd":0,"In":0,"Sn":0,
    "Sb":0,"Te":0,"I":0,"Xe":0,"Cs":0,"Ba":0,"La":0,"Ce":0,"Pr":0,"Nd":0,
    "Pm":0,"Sm":0,"Eu":0,"Gd":0,"Tb":0,"Dy":0,"Ho":0,"Er":0,"Tm":0,"Yb":0,
    "Lu":0,"Hf":0,"Ta":0,"W":0,"Re":0,"Os":0,"Ir":0,"Pt":0,"Au":0,"Hg":0,
    "Tl":0,"Pb":0,"Bi":0,"Po":0,"At":0,"Rn":0,"Fr":0,"Ra":0,"Ac":0,"Th":0,
    "Pa":0,"U":0,"Np":0,"Pu":0,"Am":0,"Cm":0,"Bk":0,"Cf":0,"Es":0,"Fm":0,
    "Md":0,"No":0,"Lr":0,"Rf":0,"Db":0,"Sg":0,"Bh":0,"Hs":0,"Mt":0}
    formula = row["Molecular Formula"]
    Z = row["Z"]
    if formula[-1] == "n":
            return 0
    parsedform = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    for element in parsedform:
        baseformula[element[0]] = float(element[1])*Z
        #print element
    return baseformula

In [ ]:
df["UnitFormula"] = df.apply(molparsed,axis=1)
df = df[df.UnitFormula !=0]
print df["UnitFormula"]

There is a new condition here, where only the outputs for UnitFormula which aren't 0 are included in the dataframe.

In [ ]:
X = []
label = []
i = 0
for x in df["UnitFormula"]:
    temp = []
    for element in x:
        temp.append(x[element])
        if i == 0:
            label.append(element)            
    X.append(temp)
    i+=1
y = df["Cell volume"]

Poorly made loop, collates the values from each row and appends the element label to it. This is a rough attempt to give the unitformula but the labeling is incorrect and the zero values should be omitted.

In [ ]:
print X, label

###### Below are examples of me trying to import and execute different models, just to get used to it.

In [ ]:
from sklearn import linear_model, svm
reg = linear_model.LinearRegression()
reg.fit(X,y)
reg.coef_

Simple ordinary least squares (OLS) linear regression.
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
regridge = linear_model.Ridge (alpha = 1)
regridge.fit (X,y)
regridge.coef_

The ridge regression is a altered version of the OLS linear regression, where an alpha parameter is used to regularize the data.
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

In [ ]:
clf = svm.SVR("linear")
clf.fit(X, y)
#clf.predict(X)
#clf.coef_
clf.score(X, y)

In [ ]:
clf1 = svm.SVR("rbf")
clf1.fit(X, y)
#clf1.predict(X)
#clf1.coef_
clf1.score(X, y)

The two above are Support Vector Regressions (SVR) using two different kernel types.
http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators=10)
clf2 = clf2.fit(X, Y)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf3 = RandomForestRegressor(n_estimators=50)
clf3.fit(X, y)

Here an attempt is made at using the random forest methods; the classifier is unnecessary for this type of analysis and was falsely imported
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor